## Server

In [ ]:
import socket
import threading
import os
import hashlib

# 미리 저장된 아이디 목록
valid_ids = ["user1", "01047034826", "01012345678"]

# 서버에서 사용할 고정 비밀 값
server_PW = "f59ecdf26cb91b476bbc9dde2fe3440ce1368cce0e73320b4066be8378dafff5"

# 클라이언트 연결을 처리하는 함수
def handle_client(client_socket, address):
    print(f"{address}에서의 연결이 성공적으로 이루어졌습니다.")
    
    # ID를 입력받음
    client_socket.send("서버에 연결되었습니다.".encode('utf-8'))
    client_id = client_socket.recv(1024).decode('utf-8').strip()
    
    if client_id in valid_ids:
        print(f"{address}의 ID: {client_id}")
        
        # 아이디 확인이 완료되었다는 메시지와 랜덤 데이터를 전송
        client_socket.send("아이디 확인이 완료되었습니다.".encode('utf-8'))
        random_data = os.urandom(16)
        client_socket.send(random_data)

        # 클라이언트로부터 OTP를 받기
        client_otp = client_socket.recv(1024).decode('utf-8')
        
        # otp유효성 확인을 위한 otp를 서버에서 생성
        server_otp = generate_server_otp(random_data)

        # 비교 후 결과를 출력하고 클라이언트에게 전송
        if server_otp == client_otp:
            print("OTP 확인이 완료되었습니다.")
            client_socket.send("OTP 확인이 완료되었습니다".encode('utf-8'))
        else:
            print("OTP가 일치하지 않습니다.")
            client_socket.send("OTP 확인에 실패하였습니다".encode('utf-8'))

        while True:
            try:
                message = client_socket.recv(1024).decode('utf-8')
                if not message:
                    break
                print(f"{client_id}({address})로부터의 메시지: {message}")
                #response = input("답변을 입력하세요: ")
                #client_socket.send(response.encode('utf-8'))
            except ConnectionResetError:
                print(f"{address}와의 연결이 강제로 종료되었습니다.")
                break
    else:
        print(f"{address}의 ID({client_id})가 유효하지 않습니다. 연결을 종료합니다.")
    
    client_socket.close()

# 서버에서 OTP 생성 함수
def generate_server_otp(random_data):
    # 클라이언트와 동일한 방식으로 OTP 생성
    otp_combined_data = bytes(x ^ y for x, y in zip(random_data, bytes.fromhex(server_PW)))
    server_otp = hashlib.sha256(otp_combined_data).hexdigest()
    print("Server OTP calculated:", server_otp)
    return server_otp

# 서버 설정
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 12347))
server_socket.listen(5)

print("서버가 포트 12347에서 대기 중입니다...")

# 스레드를 사용하여 수신된 클라이언트 연결을 처리
while True:
    client, address = server_socket.accept()
    client_handler = threading.Thread(target=handle_client, args=(client, address))
    client_handler.start()


서버가 포트 12347에서 대기 중입니다...
('127.0.0.1', 54732)에서의 연결이 성공적으로 이루어졌습니다.
('127.0.0.1', 54732)의 ID: 01012345678
Server OTP calculated: 46766c8fe60a3f40bbec5d1ecfada170fe5560e8964145b188c51041a08dbc83
OTP 확인이 완료되었습니다.
01012345678(('127.0.0.1', 54732))로부터의 메시지: exit
('127.0.0.1', 54732)와의 연결이 강제로 종료되었습니다.
